In [1]:
!pip install tensorflow==2.0.0
!pip install ISR
!pip install h5py==2.10.0

     |████████████████████████████████| 86.3 MB 11 kB/s 
     |████████████████████████████████| 449 kB 35.9 MB/s 
     |████████████████████████████████| 3.8 MB 45.7 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=9f6d3b4f39fda7599de0216ce27e1d05ee1d455a761a8b4800b724f455c53459
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Success

In [6]:
import numpy as np
from ISR.models import RDN
import zipfile
def zipdir(path, ziph):
  for root, dirs, files in os.walk(path):
    for file in files:
      ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))

In [7]:
import glob, os
os.chdir("/content")
images = []
for file in glob.glob("*.png"):
    images.append(file)


In [8]:
from PIL import Image, ImageFilter
rdn = RDN(weights='psnr-large')
baseheight = 540
for i, image in enumerate(images):
  print(str(i+1) + "/" + str(len(images)) + " "+ image)
  img = Image.open('/content/'+image)
  hpercent = (baseheight / float(img.size[1]))
  wsize = int((float(img.size[0]) * float(hpercent)))
  img = img.resize((wsize, baseheight), Image.ANTIALIAS)
  lr_img = np.array(img)
  sr_img = rdn.predict(lr_img)
  out = Image.fromarray(sr_img)
  out.save("/content/upscaled/" + images[i], "PNG")


1/42 tiao-qi.png
2/42 feng-east.png
3/42 feng-north.png
4/42 tong-er.png
5/42 tong-liu.png
6/42 tiao-san.png
7/42 feng-west.png
8/42 hua-autumn.png
9/42 hua-spring.png
10/42 tong-si.png
11/42 hua-crysanthemum.png
12/42 tiao-liu.png
13/42 tong-ba.png
14/42 tong-yi.png
15/42 wan-liu.png
16/42 tong-wu.png
17/42 tong-jiu.png
18/42 wan-ba.png
19/42 wan-jiu.png
20/42 feng-south.png
21/42 wan-si.png
22/42 hua-summer.png
23/42 hua-orchid.png
24/42 tong-qi.png
25/42 tiao-jiu.png
26/42 tiao-er.png
27/42 wan-san.png
28/42 tong-san.png
29/42 long-bai.png
30/42 long-hong.png
31/42 tiao-yi.png
32/42 wan-wu.png
33/42 tiao-si.png
34/42 long-lu.png
35/42 hua-bamboo.png
36/42 tiao-wu.png
37/42 hua-winter.png
38/42 tiao-ba.png
39/42 wan-qi.png
40/42 hua-plum.png
41/42 wan-er.png
42/42 wan-yi.png


In [9]:
zipf = zipfile.ZipFile('upscaled.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('/content/upscaled/', zipf)
zipf.close()

In [32]:
import cv2
import math
def colour_difference(r1,g1,b1, r2,g2,b2):
  return math.sqrt(math.pow(r2-r1, 2)+math.pow(g2-g1, 2)+math.pow(b2-b1, 2))

def switch_colours(img, input_colours, output_colours):
  rows, cols, _ = img.shape

  for i in range(rows):
    for j in range(cols):
        r,g,b = img.item(i,j,2), img.item(i,j,1), img.item(i,j,0)
        def diff(x):
          return colour_difference(r, g,b, x[0], x[1],x[2])
        differences = list(map(diff , input_colours))
        idx = differences.index(min(differences))
        r, g, b = output_colours[idx]
        img.itemset((i,j,2),r)
        img.itemset((i,j,1),g)
        img.itemset((i,j,0),b)



def cv2_process(img, name):
  if  name.startswith("feng"):
    # (thresh, cv2_img) = cv2.threshold(open_cv_image, 127, 255, cv2.THRESH_BINARY)
    switch_colours(img, [(34,28,23), (253,245,222)], [(0,0,0), (255,255,255)])
  elif name.startswith("hua"):
    switch_colours(img, [(34,28,23), (253,245,222), (87,92,79), (202,69,56), (198,68,77)], [(0,0,0), (255,255,255), (0,255,0), (255, 0, 0), (255,192,203)])
  elif name.startswith("long"):
    switch_colours(img, [(253,245,222), (16,12,68), (157,10,14), (0,90,30)], [(255,255,255), (0, 0 , 255), (255,0,0), (0,255,0)])
  elif name.startswith("tiao"):
    switch_colours(img, [(253,245,222), (28,19,78), (164,30,23), (1,90,30)], [(255,255,255), (0, 0 , 255), (255,0,0), (0,255,0)])
  elif name.startswith("tong"):
    switch_colours(img, [(253,245,222), (47,43,72), (145,43,44), (46,98,64)], [(255,255,255), (0, 0 , 255), (255,0,0), (0,255,0)])
  elif name.startswith("wan"):
    switch_colours(img, [(34,28,23), (253,245,222), (158,8,14)], [(0,0,0), (255,255,255), (255, 0, 0)])
    pass
  else:
    print(name)

  return img
output = []
for i, image in enumerate(images):
  img = Image.open('/content/upscaled/'+image)
  output.append(img)
for i, out in enumerate(output):
  filtered = out.filter(ImageFilter.SMOOTH_MORE).filter(ImageFilter.SMOOTH_MORE).filter(ImageFilter.BLUR).filter(ImageFilter.SHARPEN).filter(ImageFilter.ModeFilter)
  open_cv_image = np.array(filtered) 
  open_cv_image = open_cv_image[:, :, ::-1].copy() 
  cv2_img = cv2_process(open_cv_image, images[i])
  cv2.imwrite("/content/out/" + images[i], cv2_img)
  # filtered.save("/content/out/" + images[i], "PNG")

In [ ]:
from IPython.display import Image 
pil_img = Image(filename='/content/feng-east.png')
display(pil_img)

In [33]:

zipf = zipfile.ZipFile('out.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('/content/out/', zipf)
zipf.close()